In [1]:
# Imports
import pandas as pd
from glob import glob
from datetime import datetime, date, time, timedelta

In [2]:
# Read data files
list_of_data_files = glob('../data/*.txt')
list_of_data_files

['../data/turnstile_190330.txt',
 '../data/turnstile_190518.txt',
 '../data/turnstile_190525.txt',
 '../data/turnstile_190323.txt',
 '../data/turnstile_190309.txt',
 '../data/turnstile_190420.txt',
 '../data/turnstile_190427.txt',
 '../data/turnstile_190601.txt',
 '../data/turnstile_190406.txt',
 '../data/turnstile_190413.txt',
 '../data/turnstile_190511.txt',
 '../data/turnstile_190504.txt',
 '../data/turnstile_190316.txt',
 '../data/turnstile_190302.txt']

In [3]:
# Read data files into pandas dataframes
list_of_dataframes = [pd.read_csv(file, parse_dates=['DATE']) for file in list_of_data_files]
df = pd.concat(list_of_dataframes)
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-23,00:00:00,REGULAR,6989774,2370411
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-23,04:00:00,REGULAR,6989795,2370413
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-23,08:00:00,REGULAR,6989813,2370436
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-23,12:00:00,REGULAR,6989924,2370512
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-23,16:00:00,REGULAR,6990200,2370573


In [6]:
df.columns = [x.strip() for x in df.columns]
df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [ ]:
# Add a weekday column
df['WEEKDAY'] = df.apply(lambda row: row['DATE'].weekday(), axis=1)
df.head()

In [16]:
df.STATION.unique()

array(['59 ST', '57 ST-7 AV', '49 ST', 'TIMES SQ-42 ST',
       '34 ST-HERALD SQ', '28 ST', '23 ST', '8 ST-NYU', 'PRINCE ST',
       'CITY HALL', 'RECTOR ST', 'BOWERY', 'CHAMBERS ST', 'PROSPECT PARK',
       'CHURCH AV', 'BEVERLEY ROAD', 'CORTELYOU RD', 'AVENUE H',
       'AVENUE J', 'AVENUE M', 'KINGS HWY', 'NECK RD', 'BRIGHTON BEACH',
       'OCEAN PKWY', 'BOROUGH HALL', 'JAY ST-METROTEC', 'DEKALB AV',
       'ATL AV-BARCLAY', 'UNION ST', '4AV-9 ST', 'PROSPECT AV', '25 ST',
       '45 ST', '53 ST', 'BAY RIDGE AV', '77 ST', '86 ST',
       'BAY RIDGE-95 ST', '50 ST', '55 ST', '71 ST', '79 ST', '18 AV',
       '20 AV', 'W 8 ST-AQUARIUM', '6 AV', '3 AV', '1 AV', 'BEDFORD AV',
       'GRAHAM AV', 'GRAND ST', 'MONTROSE AV', 'MORGAN AV',
       'JEFFERSON ST', 'MYRTLE-WYCKOFF', 'HALSEY ST', 'WILSON AV',
       'BUSHWICK AV', 'ATLANTIC AV', 'CYPRESS HILLS', '75 ST-ELDERTS',
       '85 ST-FOREST PK', 'WOODHAVEN BLVD', '104 ST', '121 ST',
       'METROPOLITAN AV', '190 ST', '175 ST', '168 ST'

In [7]:
# Filter data to daylight hours
times = ['12:00:00', '16:00:00', '20:00:00']
df = df[df['TIME'].isin(times)]

In [8]:
df.head(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-23,12:00:00,REGULAR,6989924,2370512
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-23,16:00:00,REGULAR,6990200,2370573
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-23,20:00:00,REGULAR,6990562,2370623
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-24,12:00:00,REGULAR,6990860,2370731
10,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-24,16:00:00,REGULAR,6991054,2370788
11,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-24,20:00:00,REGULAR,6991295,2370839
15,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-25,12:00:00,REGULAR,6991651,2371197
16,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-25,16:00:00,REGULAR,6991935,2371230
17,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-25,20:00:00,REGULAR,6992684,2371283
21,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-26,12:00:00,REGULAR,6993111,2371709


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,WEEKDAY
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-23,12:00:00,REGULAR,6989924,2370512,5
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-23,16:00:00,REGULAR,6990200,2370573,5
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-23,20:00:00,REGULAR,6990562,2370623,5
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-24,12:00:00,REGULAR,6990860,2370731,6
10,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-24,16:00:00,REGULAR,6991054,2370788,6


In [11]:
# Calculate the differences between rows for entries and exits
df['EXIT_COUNT'] = df['EXITS'] - df['EXITS'].shift(1) 

In [12]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,WEEKDAY,EXIT_COUNT
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-23,12:00:00,REGULAR,6989924,2370512,5,NaN
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-23,16:00:00,REGULAR,6990200,2370573,5,61.0
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-23,20:00:00,REGULAR,6990562,2370623,5,50.0
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-24,12:00:00,REGULAR,6990860,2370731,6,108.0
10,A002,R051,02-00-00,59 ST,NQR456W,BMT,2019-03-24,16:00:00,REGULAR,6991054,2370788,6,57.0


In [ ]:
gd = df.groupby(['STATION', 'WEEKDAY', 'TIME'], as_index=False).count()

In [ ]:
gd.head()